In [1]:
# Save to sql
server = '34.125.58.101'
database = 'TestDB'
username = 'SA'
password = 'Passw0rd123456'
table_name = "demoassessment"

# 1. Profiling

In [2]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
# pd.set_option('display.max_rows', None)  # None หมายถึงไม่จำกัดจำนวนแถวที่จะแสดง


In [3]:
selectedCol_list = ['int_rate','loan_amnt']

In [4]:
#raw_df = pd.read_csv('LoanStats_web.csv', usecols=selectedCol_list)

In [5]:
import pandas as pd
from sqlalchemy import create_engine

# Establishing the connection
connection_string = f'mssql+pymssql://{username}:{password}@{server}/{database}'
engine = create_engine(connection_string)

# Reading from an existing SQL table
query = "SELECT * FROM loans_fact"
loans_fact_df = pd.read_sql(query, con=engine)

In [6]:
raw_df = loans_fact_df[selectedCol_list]

In [7]:
raw_df

,int_rate,loan_amnt
0,0.1199,12800.0000
1,0.1531,9000.0000
2,0.1075,8000.0000
3,0.1299,25000.0000
4,0.1825,35000.0000
...,...,...
1432435,0.1356,5000.0000
1432436,0.0646,6000.0000
1432437,0.1614,16000.0000
1432438,0.1131,16000.0000


In [8]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432440 entries, 0 to 1432439
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   int_rate   1432440 non-null  float64
 1   loan_amnt  1432440 non-null  float64
dtypes: float64(2)
memory usage: 21.9 MB


In [9]:
raw_df.describe(include='all')

,int_rate,loan_amnt
count,1432440.0000,1432440.0000
mean,0.1300,15370.3884
std,0.0505,9646.0263
min,0.0531,1000.0000
25%,0.0916,8000.0000
50%,0.1213,13000.0000
75%,0.1577,20000.0000
max,0.3099,40000.0000


In [10]:
#raw_df['int_rate'].str.rstrip('%').astype('float').describe()

In [11]:
# raw_df['int_rate'].str.rstrip('%').astype('float').sort_values(ascending=False)

In [12]:
raw_df.groupby('int_rate').size()

int_rate
0.0531     8613
0.0532    36953
0.0600      493
0.0607     5019
0.0608     2968
          ...  
0.3079     1223
0.3084      676
0.3089      484
0.3094      324
0.3099      464
Length: 258, dtype: int64

In [13]:
raw_df['int_rate'].value_counts()

int_rate
0.1149    49032
0.1274    41977
0.1349    39202
0.1399    39032
0.0532    36953
          ...  
0.2599        2
0.2199        2
0.1529        2
0.1319        1
0.1116        1
Name: count, Length: 258, dtype: int64

In [14]:
#raw_df['emp_length'].value_counts()

# 2. Data Rules and Use them for Measurement

In [15]:
# Define your data rules here
data_rules = {
    'int_rate': r'^\d+\.\d{2}$',
    'loan_amnt': lambda x: "NULL" if pd.isna(x) else ("FALSE" if not 1000 <= float(x) <= 40000 else "TRUE")
}

In [16]:
import re
import pandas as pd
from sqlalchemy import create_engine
import urllib


def validate_data(df, data_rules):
    """Apply regex patterns or custom functions to validate data in dataframe columns."""
    for column, rule in data_rules.items():
        if isinstance(rule, str):  # Regex pattern
            df[f'corrected_format_{column}'] = df[column].apply(
                lambda x: "NULL" if pd.isna(x) else ("TRUE" if re.match(rule, str(x)) else "FALSE")
            )
        elif callable(rule):  # Function to handle complex validations
            df[f'corrected_format_{column}'] = df[column].apply(
                lambda x: "NULL" if pd.isna(x) else rule(x)
            )
    return df

def compute_summary(df, column):
    """Print detailed summary for validation results of a specific column."""
    total_rows = df.shape[0]
    num_na = (df[f'corrected_format_{column}'] == "NULL").sum()
    num_not_na = total_rows - num_na
    num_correct = (df[f'corrected_format_{column}'] == "TRUE").sum()
    num_incorrect = (df[f'corrected_format_{column}'] == "FALSE").sum()

    print(f'Total rows: {total_rows}')
    print(f'Number of rows with {column} is NULL: {num_na}')
    print(f'Number of rows with {column} is not NULL: {num_not_na}')
    print(f'Number of rows with correct {column} format and not null: {num_correct}')
    if num_not_na > 0:  # Prevent division by zero
        print(f'Percentage of rows with correct {column} format and not null: {num_correct / num_not_na * 100:.2f}%')
    else:
        print("No non-null data available to calculate percentage of correct format.")
    print(f'Number of rows with incorrect {column} format and not null: {num_incorrect}')
    if num_not_na > 0:  # Prevent division by zero
        print(f'Percentage of rows with incorrect {column} format and not null: {num_incorrect / num_not_na * 100:.2f}%')
    print("-----------------------------------------------------------------------------------")

def save_to_sql(df, engine, table_name):
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)

In [17]:
# Implement in main script
if __name__ == "__main__":
    #filepath = 'LoanStats_web.csv'
    columns = list(data_rules.keys())
    #df = load_file(filepath, columns)
    df = validate_data(raw_df, data_rules)
    for column in data_rules.keys():
        compute_summary(df, column)
    
    
    # ตั้งค่าการเชื่อมต่อกับ MSSQL โดยใช้ข้อมูลที่เหมาะสมกับสภาพแวดล้อมของคุณ
    params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=server;DATABASE=database;UID=username;PWD=password")
    ##### engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
    # Using pymssql
    engine = create_engine(f'mssql+pymssql://{username}:{password}@{server}/{database}')
    
    save_to_sql(df, engine, table_name)


/var/folders/c5/rm_4lzt124325_03btchbfy00000gn/T/ipykernel_2530/740017105.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'corrected_format_{column}'] = df[column].apply(
/var/folders/c5/rm_4lzt124325_03btchbfy00000gn/T/ipykernel_2530/740017105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'corrected_format_{column}'] = df[column].apply(


Total rows: 1432440
Number of rows with int_rate is NULL: 0
Number of rows with int_rate is not NULL: 1432440
Number of rows with correct int_rate format and not null: 1081
Percentage of rows with correct int_rate format and not null: 0.08%
Number of rows with incorrect int_rate format and not null: 1431359
Percentage of rows with incorrect int_rate format and not null: 99.92%
-----------------------------------------------------------------------------------
Total rows: 1432440
Number of rows with loan_amnt is NULL: 0
Number of rows with loan_amnt is not NULL: 1432440
Number of rows with correct loan_amnt format and not null: 1432440
Percentage of rows with correct loan_amnt format and not null: 100.00%
Number of rows with incorrect loan_amnt format and not null: 0
Percentage of rows with incorrect loan_amnt format and not null: 0.00%
-----------------------------------------------------------------------------------
